# RDNet Rewrite

In [1]:
'''
rewrite of https://github.com/naver-ai/rdnet/blob/main/rdnet/rdnet.py in tensorflow
'''

import tensorflow as tf
from tensorflow import keras
from keras import layers, datasets
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import numpy as np

# hyperparameters
growth_k = 24
nb_block = 2
init_learning_rate = 1e-4
epsilon = 1e-4
dropout_rate = 0.2
class_num = 10


class DenseNet:
    def __init__(self, input_shape=(32, 32, 3), nb_blocks=2, filters=32, training=True):
        self.nb_blocks = nb_blocks
        self.filters = filters
        self.training = training
        self.model = self.build_network(input_shape)

    def bottleneck_layer(self, x, scope):
        x = layers.BatchNormalization(momentum=0.9, epsilon=epsilon, name=f'{scope}_batch1')(x, training=self.training)
        x = layers.ReLU()(x)
        x = layers.Conv2D(4 * self.filters, kernel_size=1, padding='same', use_bias=False, name=f'{scope}_conv1')(x)
        x = layers.Dropout(dropout_rate)(x, training=self.training)

        x = layers.BatchNormalization(momentum=0.9, epsilon=epsilon, name=f'{scope}_batch2')(x, training=self.training)
        x = layers.ReLU()(x)
        x = layers.Conv2D(self.filters, kernel_size=3, padding='same', use_bias=False, name=f'{scope}_conv2')(x)
        x = layers.Dropout(dropout_rate)(x, training=self.training)
        return x

    def transition_layer(self, x, scope):
        x = layers.BatchNormalization(momentum=0.9, epsilon=epsilon, name=f'{scope}_batch1')(x, training=self.training)
        x = layers.ReLU()(x)
        x = layers.Conv2D(int(x.shape[-1]) // 2, kernel_size=1, padding='same', use_bias=False, name=f'{scope}_conv1')(x)
        x = layers.Dropout(dropout_rate)(x, training=self.training)
        x = layers.AveragePooling2D(pool_size=(2, 2), strides=2, padding='valid')(x)
        return x

    def dense_block(self, x, nb_layers, layer_name):
        layers_concat = [x]
        for i in range(nb_layers):
            bottleneck_output = self.bottleneck_layer(x, scope=f'{layer_name}_bottleN_{i}')
            layers_concat.append(bottleneck_output)
            x = layers.Concatenate(axis=-1)(layers_concat)
        return x

    def build_network(self, input_shape):
        inputs = keras.Input(shape=input_shape)

        x = layers.Conv2D(2 * self.filters, kernel_size=7, strides=2, padding='same', use_bias=False, name='conv0')(inputs)
        x = layers.MaxPooling2D(pool_size=(3, 3), strides=2, padding='same')(x)

        for i in range(self.nb_blocks):
            x = self.dense_block(x, nb_layers=6, layer_name=f'dense_{i + 1}')
            if i != self.nb_blocks - 1:
                x = self.transition_layer(x, scope=f'trans_{i + 1}')

        x = self.dense_block(x, nb_layers=32, layer_name='dense_final')

        x = layers.BatchNormalization(momentum=0.9, epsilon=epsilon, name='linear_batch')(x, training=self.training)
        x = layers.ReLU()(x)
        x = layers.GlobalAveragePooling2D()(x)
        outputs = layers.Dense(units=class_num, activation='softmax')(x)
        return keras.Model(inputs, outputs)



2024-12-02 23:10:23.816881: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-02 23:10:23.830478: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-02 23:10:23.834494: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-02 23:10:23.845008: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

train_labels = [train_label[0] for train_label in train_labels]
test_labels = [test_label[0] for test_label in test_labels]

train_labels = np.array(train_labels, dtype=np.int8)
test_labels = np.array(test_labels, dtype=np.int8)

In [3]:
# class_names = ['plane', 'car', 'bird', 'cat', 'deer',
#                'dog', 'frog', 'horse', 'ship', 'truck']

# plt.figure(figsize=(10,10))
# for i in range(25):
#     plt.subplot(5,5,i+1)
#     plt.xticks([])
#     plt.yticks([])
#     plt.grid(False)
#     plt.imshow(train_images[i])
#     plt.xlabel(class_names[train_labels[i]])
# plt.show()

In [4]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
train_dataset = train_dataset.batch(16).prefetch(tf.data.AUTOTUNE)

test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
test_dataset = test_dataset.batch(16).prefetch(tf.data.AUTOTUNE)

I0000 00:00:1733206228.104937  115983 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:26:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1733206228.278231  115983 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:26:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1733206228.278294  115983 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:26:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1733206228.281528  115983 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:26:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1733206228.281644  115983 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:26:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

In [5]:
tf.config.optimizer.set_jit(False)  # disable xla

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

with tf.device('/GPU:0'):
    # def rdnet_tiny(pretrained=False, **kwargs):
    #     n_layer = 7
    #     model_args = {
    #         "num_init_features": 64,
    #         "growth_rates": [64] + [104] + [128] * 4 + [224],
    #         "num_blocks_list": [3] * n_layer,
    #         "is_downsample_block": (None, True, True, False, False, False, True),
    #         "transition_compression_ratio": 0.5,
    #     }
    #     model = _create_rdnet("rdnet_tiny", pretrained=pretrained, **dict(model_args, **kwargs))
    #     return model

    model = DenseNet(input_shape=(32, 32, 3), nb_blocks=5, filters=growth_k, training=True).model
    learning_rate = 0.001
    epochs = 30

    optimizer = tf.keras.optimizers.Adam(
        learning_rate=learning_rate,
    )

    model.compile(
        optimizer=optimizer,
        loss=tf.keras.losses.SparseCategoricalCrossentropy(),
        metrics=['accuracy']
    )

    history = model.fit(
        train_dataset, 
        epochs=epochs, 
        validation_data=test_dataset,
        callbacks=[EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)]
    )

Num GPUs Available:  1
Physical devices cannot be modified after being initialized
Epoch 1/30


2024-12-02 23:10:34.254061: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1228800000 exceeds 10% of free system memory.


ValueError: Exception encountered when calling AveragePooling2D.call().

[1mNegative dimension size caused by subtracting 2 from 1 for '{{node functional_1/average_pooling2d_3_1/AvgPool}} = AvgPool[T=DT_FLOAT, data_format="NHWC", ksize=[1, 2, 2, 1], padding="VALID", strides=[1, 2, 2, 1]](functional_1/dropout_51_1/stateless_dropout/SelectV2)' with input shapes: [?,1,1,138].[0m

Arguments received by AveragePooling2D.call():
  • inputs=tf.Tensor(shape=(None, 1, 1, 138), dtype=float32)

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)

In [ ]:
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label = 'val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.ylim([0, 1.75])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)